In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_market_calendars as mcal
import datetime as dt
import scipy as sp


import pymc3 as pm
import theano
from theano import tensor as tt
from theano import shared
import arviz as az
import seaborn as sns
az.style.use("arviz-darkgrid")

import os
import sys


from jupyterthemes import jtplot
jtplot.style(theme="monokai")

# Import data

In [2]:
os.chdir(sys.path[0]+"/raw_data")
os.listdir()

# Read in files and grab daily close prices
df = pd.DataFrame(data={"returns"})
for n, fname in enumerate(os.listdir()):
    if n == 0:
        ticker = fname.split(".csv")[0]
        df = pd.read_csv(fname, index_col="Date", parse_dates=True)[["Adj Close"]].rename(columns={"Adj Close" : ticker})
    else:
        ticker = fname.split(".csv")[0]
        temp_df = pd.read_csv(fname, index_col="Date", parse_dates=True)[["Adj Close"]].rename(columns={"Adj Close" : ticker})
        df = pd.merge(df, temp_df, left_index=True, right_index=True)
        
# Convert to daily log returns
df = np.log(df) - np.log(df.shift(1))
df = df.dropna()

# Input to model is each of these time series, so save
os.chdir(sys.path[0])
df.to_csv("log_returns.csv")

In [3]:
df.head()

,JNJ,LNG,MSFT,TSLA,V
Date,,,,,
2016-01-06,-0.005067,-0.037340,-0.018332,-0.019844,-0.013198
2016-01-07,-0.011723,-0.058048,-0.035402,-0.015598,-0.019858
2016-01-08,-0.010741,0.001143,0.003062,-0.021799,-0.012409
2016-01-11,-0.006029,-0.035469,-0.000574,-0.015041,0.014169
2016-01-12,0.006843,-0.009515,0.009136,0.010148,0.011299


In [4]:
data = df.values

In [5]:
nobs, ndim = data.shape

In [6]:
Sigma_a = np.random.randn(ndim, ndim)
Sigma_a = Sigma_a.T.dot(Sigma_a)
L_a = sp.linalg.cholesky(Sigma_a, lower=True)

In [7]:
data.shape

(1258, 5)

# Custom Distribution

In [18]:
from pymc3.distributions import distribution

In [35]:
class BatchMultStudentT(distribution.Continuous):
    def __init__(self, mu, nu, vols, corr, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.mu = mu
        self.vols = vols
        self.corr = corr
        self.nu = nu
        

#         self.init = init
#         self.innovArgs = (mu, cov, tau, chol, lower)
#         self.innov = multivariate.MvNormal.dist(*self.innovArgs, shape=self.shape)
#         self.mean = tt.as_tensor_variable(0.0)

    def logp(self, x):
        """
        Calculate log-probability of Multivariate Gaussian
        Random Walk distribution at specified value.
        Parameters
        ----------
        x: numeric
            Value for which log-probability is calculated.
        Returns
        -------
        TensorVariable
        """
        
        logp = 0.0
        for j in range(self.vols.shape[0]):
            vol_array = tt.nlinalg.diag(vols[j,:]).sqrt()
            cov_array = tt.nlinalg.matrix_dot(vol_array, self.corr, vol_array)
            logp += pm.MvStudentT.dist(mu=self.mu, nu=self.nu, cov=cov_array).logp(x[j,:])

        return logp

In [43]:
def batch_mult_t_logp(mu, nu, vols, corr, nobs, obs):
    '''
    Defines log probability for a batch of multivariate Student T's.
    '''
    logp = 0.0
    for j in range(nobs):
        vol_array = tt.nlinalg.diag(vols[j,:]).sqrt()
        cov_array = tt.nlinalg.matrix_dot(vol_array, corr, vol_array)
        logp += pm.MvStudentT.dist(mu=mu, nu=nu, cov=cov_array).logp(obs[j,:])

    return logp 

In [44]:
nobs = 100
obs = np.ones((nobs, 5))
batch_mult_t_logp(np.zeros(5), 3.0, np.zeros((nobs, 5)), np.identity(5), nobs,  obs)

INFO (theano.gof.compilelock): Refreshing lock C:\Users\Jonathan\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_23_Model_113_Stepping_0_AuthenticAMD-3.8.3-64\lock_dir\lock


Elemwise{add,no_inplace}.0

In [ ]:
batch_mult_t = pm.DensityDist("batch_mult_t", batch_mult_t_logp, observed={})

# Model Definition

In [8]:
def batch_mult_t_logp(mu, nu, vols, corr, nobs, obs):
    '''
    Defines log probability for a batch of multivariate Student T's.
    '''
    logp = 0.0
    for j in range(nobs):
        vol_array = tt.nlinalg.diag(vols[j,:]).sqrt()
        cov_array = tt.nlinalg.matrix_dot(vol_array, corr, vol_array)
        logp += pm.MvStudentT.dist(mu=mu, nu=nu, cov=cov_array).logp(obs[j,:])

    return logp 

with pm.Model() as model:
    
    # Model for volatility.
    vol_chol, vol_corr, vol_stds = pm.LKJCholeskyCov(
        "vol_chol", n=ndim, eta=2.0, sd_dist=pm.Exponential.dist(1.0), compute_corr=True
    )
    vol_cov = pm.Deterministic("vol_cov", vol_chol.dot(vol_chol.T))
    vols = pm.MvGaussianRandomWalk("alpha", mu=0, shape=(nobs, ndim), chol=vol_chol)
    vol_process = pm.Deterministic('vol_process', tt.exp(-2*vols))
    
    # Prior over correlation matrices
    corr_nu = pm.Uniform('corr_nu', 0, 5)
    C_triu = pm.LKJCorr('C_triu', corr_nu, ndim)
    
    # Define correlation matrix for observed returns
    C = pm.Deterministic('C', tt.fill_diagonal(C_triu[np.zeros((ndim, ndim), dtype=np.int64)], 1.))
    
    # Prior for mean of observed returns
    mu = pm.Normal("mu", 0.0, 1.0, shape=ndim)
    
    # Tail parameter
    nu1 = pm.HalfNormal("nu_minus_2", sigma=1)
    nu2 = pm.Deterministic("nu", 2.0+nu1)
    
    batch_mult_t = pm.Potential("likelihood", batch_mult_t_logp(mu, nu2, vol_process, C, nobs, data))
    #batch_mult_t = pm.DensityDist("batch_mult_t", batch_mult_t_logp, observed={"mu":mu, "nu":nu2, "vols":vol_process, "corr":C, "nobs":nobs, "obs":data})
    
#     for j in range(nobs):
#         vol_array = pm.Deterministic("vol_array_{}".format(j), tt.nlinalg.diag(vol_process[j,:]).sqrt())
#         cov_array = pm.Deterministic("cov_array_{}".format(j), tt.nlinalg.matrix_dot(vol_array, C, vol_array))
#         r = pm.MvStudentT('obs_{}'.format(j), nu=nu2, mu=mu, cov=cov_array, observed=data[j,:])

In [ ]:
with model:
    trace = pm.sample(1000, cores=1)

C:\Users\Jonathan\anaconda3\lib\site-packages\pymc3\sampling.py:465: FutureWarning: In an upcoming release, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  warnings.warn(
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


In [30]:
theano.config.compute_test_value = "ignore"

In [32]:
x = tt.dvector('x')
y = tt.dvector('y')

def fun(x,a,b):
    return x+b

results, updates = theano.scan(
    fn=fun,
    sequences=dict(input=x), 
    outputs_info=dict(initial=y, taps=[-5,-3])
)

h = [10.,20,30,40,50,60,70]
f = theano.function([x, y], results)
g = theano.function([y], y)

print(f([1],h))

[31.]


# Theano scan function notes
outputs, updates = theano.scan(
    fn = some_function,
    sequences = thing_to_loop_over,
    n_steps = number_of_times_to_iterate
)

In [33]:
x = tt.vector("x")

def square(x):
    return x*x

outputs, updates = theano.scan(
    fn = square,
    sequences = x,
    n_steps = x.shape[0]
)

square_op = theano.function(
    inputs = [x],
    outputs = [outputs]
)

o_val = square_op(np.array([1, 2, 3, 4, 5]))

INFO (theano.gof.compilelock): Refreshing lock C:\Users\Jonathan\AppData\Local\Theano\compiledir_Windows-10-10.0.19041-SP0-AMD64_Family_23_Model_113_Stepping_0_AuthenticAMD-3.8.3-64\lock_dir\lock


In [34]:
o_val

[array([ 1.,  4.,  9., 16., 25.])]

In [ ]:
result = tt.vector("x")

def square(prev_sum,):
    return x*x

outputs, updates = theano.scan(
    fn = square,
    sequences = x,
    n_steps = x.shape[0]
)

square_op = theano.function(
    inputs = [x],
    outputs = [outputs]
)

o_val = square_op(np.array([1, 2, 3, 4, 5]))

In [40]:
pm.MvGaussianRandomWalk.dist(mu=np.zeros((nobs, ndim)), shape=(nobs, ndim), chol=L_a).random()

AttributeError: 'MvGaussianRandomWalk' object has no attribute 'random'

In [41]:
from pymc3.distributions.timeseries import MvGaussianRandomWalk

In [44]:
with pm.Model():
    mu = np.array([1.0, 0.0])
    cov = np.array([[1.0, 0.0],
                    [0.0, 2.0]])
    sample = MvGaussianRandomWalk(mu, cov, shape=(10, 2)).random()

TypeError: 'NoneType' object is not callable

In [47]:
sample = MvGaussianRandomWalk.dist(mu, cov, shape=(10, 2)).random()

AttributeError: 'MvGaussianRandomWalk' object has no attribute 'random'

In [ ]:

            with pm.Model():
                mu = np.array([1.0, 0.0])
                cov = np.array([[1.0, 0.0],
                                [0.0, 2.0]])
                # draw one sample from a 2-dimensional Gaussian random walk with 10 timesteps
                sample = MvGaussianRandomWalk(mu, cov, shape=(10, 2)).random()
                # draw three samples from a 2-dimensional Gaussian random walk with 10 timesteps
                sample = MvGaussianRandomWalk(mu, cov, shape=(10, 2)).random(size=3)
                # draw four samples from a 2-dimensional Gaussian random walk with 10 timesteps,
                # indexed with a (2, 2) array
                sample = MvGaussianRandomWalk(mu, cov, shape=(10, 2)).random(size=(2, 2))
        """

In [ ]:
  #packed_L_a = pm.LKJCholeskyCov("packed_L_a", n=ndim, eta=2.0, sd_dist=pm.HalfCauchy.dist(2.5))
    #L_a = pm.expand_packed_triangular(ndim, packed_L_a)
    #a = pm.MvGaussianRandomWalk("alpha", mu=0, shape=(nobs, ndim), chol=L_a)
    #vol_process = pm.Deterministic('vol_process', tt.exp(-2*a))
    
    # Correlation in observed returns (independent of volatility)